$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bm}[1]{{\bf #1}}
\newcommand{\bb}[1]{\bm{\mathrm{#1}}}
$$

# Our Project
<a id=part2></a>

imports and stuff

In [1]:
import unittest
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile

import numpy as np
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


### Obtaining the dataset
<a id=part2_1></a>

Getting it from the specific folder
Create a `Dataset` object that will load the extraced images:

In [3]:
import hw3.cs236781.plot as plot
import torchvision.transforms as T
from torchvision.datasets import ImageFolder

dataset_dir = './imgs/CelebAMask-HQ/CelebA-HQ-img'
im_size = 1024
tf = T.Compose([
    # Resize to constant spatial dimensions
    T.Resize((im_size, im_size)),
    # PIL.Image -> torch.Tensor
    T.ToTensor(),
    # Dynamic range [0,1] -> [-1, 1]
    T.Normalize(mean=(.5,.5,.5), std=(.5,.5,.5)),
])

ds_gwb = ImageFolder(os.path.dirname(dataset_dir), tf)

OK, let's see what we got. You can run the following block multiple times to display a random subset of images from the dataset.

In [4]:
from hw3.Indicators import IndicatorsResNet50
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from hw3.training import IndicatorTrainer
from hw3.data_loader import ToTensor, CelebA

# Data
ds = CelebA('./imgs/CelebAMask-HQ/CelebA-HQ-img', 
            './imgs/CelebAMask-HQ/CelebA-HQ-img/annotation_indicators.csv',
            './imgs/CelebAMask-HQ/CelebAMask-HQ-mask-anno/CelebAMask-HQ-mask-anno-complete',
            './imgs/CelebAMask-HQ/CelebAMask-HQ-mask-anno/CelebAMask-HQ-mask-anno-complete/sized_and_centers.csv', ToTensor())
ds_train, ds_test = random_split(ds, [25000, 5000])
#ds_train = CelebAIndicator(r"C:\Users\Gchiko\PycharmProjects\DeepLearningProject\NoteBook\imgs\CelebA-HQ-img-Train", r"C:\Users\Gchiko\PycharmProjects\DeepLearningProject\NoteBook\imgs\CelebA-HQ-img-Train\annotation_indicators.csv", ToTensor())
#ds_test = CelebAIndicator(r"C:\Users\Gchiko\PycharmProjects\DeepLearningProject\NoteBook\imgs\CelebA-HQ-img-Test", r"C:\Users\Gchiko\PycharmProjects\DeepLearningProject\NoteBook\imgs\CelebA-HQ-img-Test\annotation_indicators.csv", ToTensor())

dl_train = DataLoader(ds_train, 125, shuffle=True, num_workers=1)
dl_test  = DataLoader(ds_test,  125, shuffle=True, num_workers=1)

# Model
ind_model = IndicatorsResNet50(10)

# Optimizer
optimizer = optim.Adam(ind_model.parameters(), lr=0.01)

# Loss
loss = torch.nn.BCEWithLogitsLoss()

#Trainer
trainer = IndicatorTrainer(ind_model, loss, optimizer, 'cpu')

# Show model and hypers
print(ind_model)

IndicatorsResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [5]:
import IPython.display

def post_epoch_fn(epoch, train_result, test_result, verbose):
    print(f"epoch num:{epoch}/ntrain result:{train_result}/ntest_result:{test_result}/n")
        
res = trainer.fit(dl_train, dl_test,
                      num_epochs=100, checkpoints='checkpoint_file', early_stopping=15, print_every=2)

--- EPOCH 1/100 ---
train_batch:   0%|                                                                             | 0/200 [00:23<?, ?it/s]


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 8388608000 bytes. Buy new RAM!
